**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Notebook 2 - Open Science Platform - User Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a user account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the user account;

6 - Queries the blockchain again to confirm the succesfull details setting.

User accounts and their respective attributes are randomly picked, every account succesfully created is dumped to `datasets/accounts.json` for later use and evidence collection.  

In [46]:
from name_generator import left, right
from utilities import *
from dump_to_json import *
from iroha_helper import *
from new_helper import extract_account_metadata_cid_from_data, process_raw_data, load_variable
from loguru import logger

In [47]:
DOMAIN = "test"

1 - Loads the smart contract hash for account creation into the Iroha 1 blockchain.

In [48]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)


2025-03-04 01:42:55,776 - INFO - Variable 'Create Account Hash' successfully loaded from temp/Create Account Hash.json
2025-03-04 01:42:55.778 | INFO     | __main__:<module>:3 - b'8e084bc80f005ae06e30255a1bb1b28abecdc3da9d6533ab4751fa58e4a8a0ae'


2 - Creates a user account into the blockchain.

In [49]:
user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key).decode("utf-8")

# - Random generated synthetic user data 
user_account_left = random.choice(left)
user_account_right = random.choice(right)
user_account_short_id = f"{user_account_left}_{user_account_right}"
user_account_full_name = ((f"{user_account_left}").capitalize())+" "+((f"{user_account_right}").capitalize())
user_account_email = f"{user_account_left}_{user_account_right}"+"@email.com"
user_account_institution = print_random_from_second_column("datasets/world-universities.csv")
user_account_orcid = generate_orcid()
user_role = set_random_role()
logger.info(f"User Role: {user_role}")

user_account = UserAccount(
    account_id=f"{user_account_short_id}@{DOMAIN}",
    full_name=user_account_full_name,
    email=user_account_email,
    institution=user_account_institution,
    orcid=user_account_orcid,
    role=user_role,
    public_key=user_public_key,
)
logger.info(f"Creating account with name: {user_account_short_id}")

# - Create account
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)
create_user_account(create_account_contract_address, user_account_short_id, DOMAIN, user_public_key, user_account)

2025-03-04 01:42:55.834 | INFO     | __main__:<module>:13 - User Role: reviewer
2025-03-04 01:42:55.836 | INFO     | __main__:<module>:24 - Creating account with name: zealous_ptolemy
2025-03-04 01:42:55.837 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-03-04 01:42:55.855 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'8e084bc80f005ae06e30255a1bb1b28abecdc3da9d6533ab4751fa58e4a8a0ae'
2025-03-04 01:42:55.857 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 1A080D809C95C764FC96320885CFAF7744ED8970
2025-03-04 01:42:55.858 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-03-04 01:42:55.860 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_user_account"
2025-03-04 01:42:55,879 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-03-04 01:42:55,881 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)

3 - Queries the blockchain to confirm the succesful creation of the account.

In [50]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
address = integration_helpers.get_engine_receipts_result(get_account_hash)

2025-03-04 01:42:57.649 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-03-04 01:42:57,651 - WARNING - create_account_contract_address: 1A080D809C95C764FC96320885CFAF7744ED8970
2025-03-04 01:42:57,668 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-03-04 01:42:57,670 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-03-04 01:43:00,657 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-03-04 01:43:00,667 - INFO - ('COMMITTED', 5, 0)
2025-03-04 01:43:00,669 - WARNING - Account Data: {'address': '1A080D809C95C764FC96320885CFAF7744ED8970', 'hex_hash': 'bcab829c4006a5d161969283be179b7a7652e022cfd0f84688df38c206bca1d9', 'timestamp': 1741052580669}
2025-03-04 01:43:00.670 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-03-04 01:43:00,673 - INFO - Appended data under 'zealous_ptolemy@test' in logs/account_data.json
2025-03-04 01:43:00.674 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-03-04 01

4 - Loads the smart contract for details (attributes) setting into the Iroha 1 blockchain .

In [51]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-03-04 01:43:00,713 - INFO - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-03-04 01:43:00.715 | INFO     | __main__:<module>:3 - b'd106deced13a349403f823cc50875ad0ad36ac062b6da6b258e227fea9280824'


5 - Sets details for the user account:

- Full Name
- Email
- Institution
- ORCID
- Public key
- Role

In [52]:
address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)

process_account(address, f"{user_account_short_id}@{DOMAIN}")

2025-03-04 01:43:00.726 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-03-04 01:43:00.748 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'd106deced13a349403f823cc50875ad0ad36ac062b6da6b258e227fea9280824'
2025-03-04 01:43:00.749 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: B1665A25BB86FD84BF06FAC9980EC563FDBFC55D
2025-03-04 01:43:00.750 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-03-04 01:43:00.751 | DEBUG    | integration_helpers:tracer:35 - 	Entering "process_account"
2025-03-04 01:43:00,753 - INFO - Processing account: zealous_ptolemy@test
2025-03-04 01:43:00,754 - INFO - User Account Metadata: {'@type': 'foaf:Person', 'foaf:name': 'Zealous Ptolemy', 'foaf:mbox': 'zealous_ptolemy@email.com', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Ashkelon Academic College'}, 'schem

6 - Queries the blockchain again to confirm the succesfull details setting.

In [53]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
details = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
account_metadata_cid, project_id = process_raw_data(details)

2025-03-04 01:43:03.732 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-03-04 01:43:03,735 - WARNING - create_account_contract_address: 1A080D809C95C764FC96320885CFAF7744ED8970
2025-03-04 01:43:03,750 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-03-04 01:43:03,752 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-03-04 01:43:06,724 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-03-04 01:43:06,732 - INFO - ('COMMITTED', 5, 0)
2025-03-04 01:43:06,734 - WARNING - Account Data: {'address': '1A080D809C95C764FC96320885CFAF7744ED8970', 'hex_hash': 'bab70cdeed74aed0929e7b6f68c4890c67cb790853f6f22c0ec585aa3c02c263', 'timestamp': 1741052586734}
2025-03-04 01:43:06.736 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-03-04 01:43:06,739 - INFO - Appended data under 'zealous_ptolemy@test' in logs/account_data.json
2025-03-04 01:43:06.740 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-03-04 01

7 - Retrieves User Metadata (JSON-LD formatted) from IPFS

In [54]:
user_metadata = download_json_from_ipfs(account_metadata_cid)


2025-03-04 01:43:06.791 | SUCCESS  | ipfs_functions:download_json_from_ipfs:52 - Success downloading JSON from IPFS. Metadata: {'@type': 'foaf:Person', 'foaf:holdsAccount': {'schema:identifier': 'zealous_ptolemy@test', 'schema:publicKey': 'ca4c00c0a43bbd2caf070ab780886906ebb70e2c3d975972ccab4e15c01f33bd', 'schema:roleName': 'reviewer'}, 'foaf:mbox': 'zealous_ptolemy@email.com', 'foaf:name': 'Zealous Ptolemy', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Ashkelon Academic College'}, 'schema:identifier': {'@type': 'PropertyValue', 'propertyID': 'ORCID', 'value': '6153-7096-0437-X'}}
